In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from tqdm import tqdm 
import numpy as np

from committee_learning.time_measure import expected_exit_time
from committee_learning.initial_conditions import RandomNormalInitialConditions
from committee_learning.simulation import NormalizedSphericalConstraintSimulation
from committee_learning.result import FullODEResult
from committee_learning.ode import SphericalSquaredActivationODE

from committee_learning.time_measure import exit_time

from committee_learning.sde.generalized_phase_retrieval import SphericalGeneralizedPhaseRetrievalSDE

n_ids = 10 # This is the number of samples used for the time measure.
# This is just an example that should run fast; 10 is too small for an accurate measure.
save_per_decade = 5000
log_time = 1
noise = 1e-4
dt = 1e-3

k = 1
p = 1
d = 3000
gamma = .1
alpha = gamma / p
T = .01
disable_QM_save=True
icid = 9

ic_string = 'randinint_{icid}'.format(icid=icid)

ics = RandomNormalInitialConditions(p, k, d, seed = icid, spherical=True)
exit_times_sde = expected_exit_time(
    SphericalGeneralizedPhaseRetrievalSDE,
    gamma = gamma,
    gamma_over_p = gamma/p,
    noise = noise,
    ic = ics,
    T = T,
    log_time = log_time,
    dt = dt,
    activation='square',
    path='computation-database/time-measure/sde/',
    save_per_decade = save_per_decade,
    ids = n_ids,
    icid = ic_string,
    disable_QM_save= disable_QM_save,
    # allow_missing=True,
    quadratic_terms=True,
    xor_seed=icid,
    noise_term = True
)


exit_times_sim = expected_exit_time(
    NormalizedSphericalConstraintSimulation,
    gamma = gamma,
    noise = noise,
    ic = ics,
    T = T,
    log_time = log_time,
    activation='square',
    path='computation-database/time-measure/sim/',
    save_per_decade = save_per_decade,
    ids = n_ids,
    icid = ic_string,
    disable_QM_save= disable_QM_save,
    xor_seed=icid,
)

ode = SphericalSquaredActivationODE(
    P0 = ics.P, Q0=ics.Q, M0=ics.M,
    dt=dt, noise_term=True, gamma_over_p=gamma/p, noise=noise,
    quadratic_terms=True, disable_QM_save= disable_QM_save
)
oder = FullODEResult(
    initial_condition='time-measure'+'ODE'+ic_string,
    id = 0
)
oder.from_file_or_run(
    ode,
    decades = log_time,
    path='computation-database_pd/time-measure/ode/',
    show_progress=True
)

et_ode = exit_time(
    np.array(oder.times),
    np.array(oder.risks),
    T
)

et_sde, et_sde_err = exit_times_sde
et_sim, et_sim_err = exit_times_sim

In [18]:
print(f'Exit time masured from simulations is {et_sim:.3f}±{et_sim_err/np.sqrt(n_ids):.3f}')
print(f'Exit time masured from SDEs is {et_sde:.3f}±{et_sde_err/np.sqrt(n_ids):.3f}')
print(f'Exit time measured from ODE is {et_ode:.3f}')

Exit time masured from simulations is 1.327±0.252
Exit time masured from SDEs is 0.942±0.095
Exit time measured from ODE is 1.091
